In [1]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [2]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

<gzip _io.BufferedReader name='./ukb-b-6134.vcf.gz' 0x29cc7f54b38>


In [3]:
import os
import json
import pickle
import time
import gzip
import shutil

def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
    #     print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [4]:
# generate_gwas_output_as_tsv_file(inVCF_File='./ukb-b-6134.vcf.gz', writeFile=None)

In [5]:

# ' abc def\n'.strip()

In [6]:
# 10**(-1*0.200659) #0.6300007

In [7]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
print(df['Category'].unique())
GWAS_ID = 'ieu-b-111'
category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
category

['Continuous' 'Binary' 'Risk factor' 'NA (Possibly binary)'
 'Categorical Ordered (assumed continuous)']


'beta'

In [8]:
from glob import glob
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
ALL_GWAS_IDS = sorted(list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257'}))
base_files = {}
for GWAS_ID in ALL_GWAS_IDS:
    print(GWAS_ID)
    file = ROOT + GWAS_ID + '.vcf.gz'
    base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)

ieu-b-109
ieu-b-110
ieu-b-111
ieu-b-25
ieu-b-38
ieu-b-39
met-d-Total_C
ukb-b-12064
ukb-b-13806
ukb-b-17627
ukb-b-18275
ukb-b-19953
ukb-b-2209
ukb-b-3957
ukb-b-4424
ukb-b-6134
ukb-b-6324
ukb-b-7663
ukb-b-770
ukb-b-8476
ukb-d-20405_0
ukb-d-20405_1
ukb-d-20405_2


In [9]:
base_files

{'ieu-b-109': 'beta',
 'ieu-b-110': 'beta',
 'ieu-b-111': 'beta',
 'ieu-b-25': 'beta',
 'ieu-b-38': 'beta',
 'ieu-b-39': 'beta',
 'met-d-Total_C': 'beta',
 'ukb-b-12064': 'or',
 'ukb-b-13806': 'or',
 'ukb-b-17627': 'beta',
 'ukb-b-18275': 'or',
 'ukb-b-19953': 'beta',
 'ukb-b-2209': 'beta',
 'ukb-b-3957': 'beta',
 'ukb-b-4424': 'beta',
 'ukb-b-6134': 'beta',
 'ukb-b-6324': 'beta',
 'ukb-b-7663': 'or',
 'ukb-b-770': 'beta',
 'ukb-b-8476': 'or',
 'ukb-d-20405_0': 'or',
 'ukb-d-20405_1': 'or',
 'ukb-d-20405_2': 'or'}

In [10]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

# NUM_TRAINING_SAMPLES = int(830 * .7)
# print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
# f_writable = open('./COVAR_FILE.txt', 'w')
# f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
# with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
#     for i, line in enumerate(f):
#         if 'FID IID' in line:
#             line = line[:-1] + ' GENDER AGE\n'
#         else:
#             line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
#         if i < NUM_TRAINING_SAMPLES:
#             f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
#         f_writable.write(line)
# #         print(line)
# f_writable.close()

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18,19,20,103,104) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
def run_prsice(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
    return_status = os.system(f'mkdir ./prsice_output/{GWAS_ID}/')
    print('0:', return_status)
    if return_status == 256 and not redo:
        return return_status
    prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
        --dir ./prsice_output \
        --prsice ./PRSice_linux/PRSice_linux \
        --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
        --target {TARGET_DATA} \
        --thread 64 \
        --stat {base_files[GWAS_ID].upper()} \
        --{base_files[GWAS_ID]} \
        --binary-target F \
        --quantile 10 \
        --out ./prsice_output/{GWAS_ID}/{GWAS_ID} \
        --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
        --score std \
        --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt'
    print(prsice_command)
#     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1a:', return_status)
#     return
    if os.path.isfile(f'./prsice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
        prsice_command += f'  --extract ./prsice_output/{GWAS_ID}/{GWAS_ID}.valid' 
#         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
        return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
        print('1b:', return_status)
        if return_status != 0:
            return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
            print('1c:', return_status)
#     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
#     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
#     print('2:', return_status)
    # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')

In [12]:
for GWAS_ID in base_files:
    print(GWAS_ID)
#     run_prsice(GWAS_ID=GWAS_ID)
#     break

ieu-b-109
ieu-b-110
ieu-b-111
ieu-b-25
ieu-b-38
ieu-b-39
met-d-Total_C
ukb-b-12064
ukb-b-13806
ukb-b-17627
ukb-b-18275
ukb-b-19953
ukb-b-2209
ukb-b-3957
ukb-b-4424
ukb-b-6134
ukb-b-6324
ukb-b-7663
ukb-b-770
ukb-b-8476
ukb-d-20405_0
ukb-d-20405_1
ukb-d-20405_2


In [13]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [14]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
base_files
GWAS_ID = 'ieu-b-109'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./prsice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [16]:
get_prs_values(GWAS_ID=GWAS_ID).shape
base_files

{'ieu-b-109': 'beta',
 'ieu-b-110': 'beta',
 'ieu-b-111': 'beta',
 'ieu-b-25': 'beta',
 'ieu-b-38': 'beta',
 'ieu-b-39': 'beta',
 'met-d-Total_C': 'beta',
 'ukb-b-12064': 'or',
 'ukb-b-13806': 'or',
 'ukb-b-17627': 'beta',
 'ukb-b-18275': 'or',
 'ukb-b-19953': 'beta',
 'ukb-b-2209': 'beta',
 'ukb-b-3957': 'beta',
 'ukb-b-4424': 'beta',
 'ukb-b-6134': 'beta',
 'ukb-b-6324': 'beta',
 'ukb-b-7663': 'or',
 'ukb-b-770': 'beta',
 'ukb-b-8476': 'or',
 'ukb-d-20405_0': 'or',
 'ukb-d-20405_1': 'or',
 'ukb-d-20405_2': 'or'}

In [17]:
import numpy as np

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = {}
PRS_feature_matrix = np.zeros([len(base_files), 1816])
for i, GWAS_ID in enumerate(base_files):
    print(i, GWAS_ID)
    PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits[GWAS_ID] = trait
print(all_traits)
json.dump(all_traits, open('all_traits_map.json', 'w'))

0 ieu-b-109
ieu-b-109 : HDL cholesterol
1 ieu-b-110
ieu-b-110 : LDL cholesterol
2 ieu-b-111
ieu-b-111 : triglycerides
3 ieu-b-25
ieu-b-25 : Cigarettes per Day
4 ieu-b-38
ieu-b-38 : systolic blood pressure
5 ieu-b-39
ieu-b-39 : diastolic blood pressure
6 met-d-Total_C
met-d-Total_C : Total cholesterol
7 ukb-b-12064
ukb-b-12064 : Non-cancer illness code, self-reported: depression
8 ukb-b-13806
ukb-b-13806 : Non-cancer illness code, self-reported: type 2 diabetes
9 ukb-b-17627
ukb-b-17627 : Non-oily fish intake
10 ukb-b-18275
ukb-b-18275 : Hearing difficulty/problems with background noise
11 ukb-b-19953
ukb-b-19953 : Body mass index (BMI)
12 ukb-b-2209
ukb-b-2209 : Oily fish intake
13 ukb-b-3957
ukb-b-3957 : Sleeplessness / insomnia
14 ukb-b-4424
ukb-b-4424 : Sleep duration
15 ukb-b-6134
ukb-b-6134 : Age completed full time education
16 ukb-b-6324
ukb-b-6324 : Processed meat intake
17 ukb-b-7663
ukb-b-7663 : Types of physical activity in last 4 weeks: Strenuous sports
18 ukb-b-770
ukb-b-7

In [18]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save('PRS_feature_matrix', PRS_feature_matrix)

In [19]:
PRS_feature_matrix.shape
import random

In [20]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(847))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[580, 736, 466, 600, 826, 617, 750, 827, 659, 126, 406, 30, 363, 112, 110, 39, 644, 370, 542, 768, 214, 266, 522, 166, 327, 775, 760, 114, 345, 664, 32, 761, 846, 218, 326, 684, 398, 779, 99, 277, 824, 704, 281, 429, 498, 225, 523, 734, 287, 716, 263, 473, 698, 596, 111, 469, 778, 672, 390, 37, 167, 573, 324, 361, 415, 58, 393, 365, 802, 271, 65, 251, 489, 490, 478, 553, 375, 313, 230, 268, 401, 656, 815, 633, 773, 169, 840, 234, 711, 667, 262, 722, 233, 301, 451, 340, 514, 605, 702, 249, 274, 343, 123, 607, 173, 749, 632, 758, 739, 240, 647, 59, 149, 36, 674, 675, 226, 64, 475, 843, 392, 767, 488, 434, 650, 806, 660, 72, 718, 24, 646, 808, 427, 710, 669, 708, 10, 46, 835, 845, 641, 738, 374, 455, 256, 316, 752, 823, 174, 436, 456, 540, 438, 285, 591, 622, 569, 303, 20, 45, 602, 730, 23, 95, 5, 157, 505, 740, 562, 312, 404, 559, 176, 141, 7, 476, 250, 376, 62, 534, 418, 221, 747, 125, 790, 552, 172, 484, 194, 748, 188, 397, 811, 700, 16, 686, 38, 100, 339, 185, 751, 171, 150, 40, 593, 

(array([ 0.02499309, -0.08205707,  0.00948655,  0.02489173,  0.04579776,
        -0.00239782, -0.08748314, -0.18570752, -0.05093119,  0.02142892,
        -0.58405674,  0.00712007, -0.06555401, -0.01525316,  0.11998356,
         0.02873634,  0.02805825, -0.55428763,  0.02592091, -0.67180425,
        -0.51258025, -0.23649956, -0.50187668]),
 array([1.00386333, 0.9999153 , 0.95658221, 0.97260175, 0.99413015,
        0.99916697, 1.02843467, 0.90026976, 0.97299565, 0.93790211,
        0.57573282, 1.03492791, 0.99948307, 0.94893407, 0.67925957,
        1.01930569, 1.01136543, 0.55415363, 1.00188153, 0.69361279,
        0.57591682, 0.8644867 , 0.50885885]))

In [21]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([ 0.05863466, -0.0713431 , -0.06721576, -0.04784146, -0.03317632,
        -0.00111891, -0.035894  , -0.1540796 , -0.1255388 , -0.02723909,
        -0.5873741 , -0.02917095,  0.04124887,  0.06374563,  0.13103953,
        -0.05948543, -0.03217983, -0.59134381,  0.03948564, -0.67172961,
        -0.53448907, -0.26734047, -0.4879381 ]),
 array([0.98856072, 0.93626003, 0.93096018, 0.95218155, 0.92144264,
        0.97391392, 1.04672332, 1.01298602, 0.99465102, 1.00665934,
        0.59205417, 0.95295485, 0.99139386, 0.94918448, 0.68056723,
        0.96937971, 0.94538941, 0.55401544, 0.96683177, 0.71203078,
        0.58041407, 0.87320802, 0.53227116]))

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [22]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

(array([ 8.02312304e-12, -3.06668445e-11,  6.21563867e-11, -2.72191667e-11,
         7.99063801e-12,  7.19383305e-11,  1.19856821e-11, -3.74664176e-11,
         1.80121097e-11,  1.85110057e-11, -5.24669451e-11,  1.11084759e-11,
         5.59416282e-11,  3.21943862e-11, -5.24559427e-11, -2.76734485e-12,
        -1.95044059e-11,  9.72282264e-12,  5.42070158e-12,  6.10582832e-11,
        -7.23843658e-11, -4.03398744e-11, -1.24741255e-11]),
 array([0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463]))

In [23]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./prsice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {'_'.join(sample.split('_')[1:]):idx for idx,sample in enumerate(usable_samples_ADNI)}
# json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
type(usable_samples_ADNI), len(usable_samples_ADNI)

(dict, 1816)

In [24]:
!ls prsice_output

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
import json
import numpy as np
PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
# usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

usable_samples_ADNI

(1816, 23) 1816


{'036_S_6231': 0,
 '006_S_6277': 1,
 '129_S_6146': 2,
 '033_S_6352': 3,
 '027_S_6183': 4,
 '005_S_6427': 5,
 '127_S_6147': 6,
 '114_S_6251': 7,
 '129_S_6228': 8,
 '114_S_6309': 9,
 '135_S_6110': 10,
 '020_S_6358': 11,
 '135_S_6411': 12,
 '024_S_6202': 13,
 '018_S_6414': 14,
 '002_S_6103': 15,
 '177_S_6408': 16,
 '014_S_6148': 17,
 '036_S_6466': 18,
 '036_S_6134': 19,
 '007_S_6455': 20,
 '037_S_6271': 21,
 '116_S_6100': 22,
 '027_S_6327': 23,
 '099_S_6097': 24,
 '127_S_6330': 25,
 '127_S_6168': 26,
 '018_S_6351': 27,
 '009_S_6212': 28,
 '168_S_6180': 29,
 '116_S_6119': 30,
 '023_S_6346': 31,
 '168_S_6065': 32,
 '035_S_6200': 33,
 '023_S_6399': 34,
 '037_S_6125': 35,
 '116_S_6428': 36,
 '041_S_6192': 37,
 '941_S_6333': 38,
 '006_S_6209': 39,
 '033_S_6497': 40,
 '127_S_6436': 41,
 '141_S_6178': 42,
 '002_S_6456': 43,
 '168_S_6062': 44,
 '033_S_6298': 45,
 '002_S_6053': 46,
 '135_S_6359': 47,
 '141_S_6015': 48,
 '027_S_6317': 49,
 '002_S_6009': 50,
 '098_S_6343': 51,
 '024_S_6033': 52,
 '9

In [1]:
THRESHOLD_MONTH = 12*4
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
Samples_Dementia = set(Dementia['PTID'].unique())
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH or True:
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
                Final_Samples_Dementia = Final_Samples_Dementia.union({sample})

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
    if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {sample} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
json.dump(Final_Samples, open('Final_Samples.json', 'w'))

Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__(), Final_Samples_Dementia.__len__()/Final_Samples.__len__(), Final_Samples_NonDementia.__len__()/Final_Samples.__len__()

NameError: name 'pd' is not defined

In [27]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./prsice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))

In [28]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

['ieu-b-109', 'ieu-b-110', 'ieu-b-111', 'ieu-b-25', 'ieu-b-38', 'ieu-b-39', 'met-d-Total_C', 'ukb-b-12064', 'ukb-b-13806', 'ukb-b-17627', 'ukb-b-18275', 'ukb-b-19953', 'ukb-b-2209', 'ukb-b-3957', 'ukb-b-4424', 'ukb-b-6134', 'ukb-b-6324', 'ukb-b-7663', 'ukb-b-770', 'ukb-b-8476', 'ukb-d-20405_0', 'ukb-d-20405_1', 'ukb-d-20405_2']


In [29]:
df

,GWAS_ID,Trait,Category
0,bbj-a-46,Pulse Pressure,Continuous
1,bbj-a-78,Smoking behaviors : Smoking initiation,Binary
2,ukb-b-13806,"Non-cancer illness code, self-reported: type 2...",Binary
3,ieu-a-1283,Alcohol consumption,Risk factor
4,ukb-d-20405_0,"Ever had known person concerned about, or reco...",Binary
5,ukb-d-20405_1,"Ever had known person concerned about, or reco...",Binary
6,ukb-d-20405_2,"Ever had known person concerned about, or reco...",Binary
7,met-d-Total_C,Total cholesterol,Continuous
8,ieu-b-109,HDL cholesterol,Continuous
9,ieu-b-110,LDL cholesterol,Continuous


In [30]:
# df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# all_traits = {}
# for GWAS_ID in ALL_GWAS_IDS:
#     trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
#     print(GWAS_ID, ':', trait)
#     all_traits[GWAS_ID] = trait
# print(all_traits)

In [31]:

import torch
torch.__version__

'1.6.0'

In [32]:
list(all_traits)
[all_traits[x] for x in all_traits]

['HDL cholesterol',
 'LDL cholesterol',
 'triglycerides',
 'Cigarettes per Day',
 'systolic blood pressure',
 'diastolic blood pressure',
 'Total cholesterol',
 'Non-cancer illness code, self-reported: depression',
 'Non-cancer illness code, self-reported: type 2 diabetes',
 'Non-oily fish intake',
 'Hearing difficulty/problems with background noise',
 'Body mass index (BMI)',
 'Oily fish intake',
 'Sleeplessness / insomnia',
 'Sleep duration',
 'Age completed full time education',
 'Processed meat intake',
 'Types of physical activity in last 4 weeks: Strenuous sports',
 'Other meat intake',
 'Loneliness, isolation',
 'Ever had known person concerned about, or recommend reduction of, alcohol consumption: No',
 'Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, but not in the last year',
 'Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, during the last year']